In [1]:
import datetime
import json
# from pyspark import SparkContext
# from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier


In [2]:
events = sc.textFile('s3a://meetupevents/fix2017/*/*/*/*')

In [4]:
events.take(2)

[u'{"utc_offset":7200000,"venue":{"zip":"meetup136","country":"nl","city":"Meppel","address_1":"Tuinweg","name":"Plek van Erkenning","lon":6.188538,"lat":52.680996},"rsvp_limit":0,"venue_visibility":"public","visibility":"public","maybe_rsvp_count":0,"description":"<p>Kies voor een Reiki behandeling of een intuitieve healing. Ik help je graag op je pad naar een leven waarin je alleen maar hoeft te volgen waar je blij van wordt. Soms betekent dit eerst het loslaten van oude patronen of het genezen van (oude) wonden. Energetische healing is hierbij een stukje van de puzzel en zal je precies geven wat jij op dit moment kan gebruiken. Je kan een healing boeken op elke dag van de week. Dag of avond. Ik heb een locatie in IJhorst en Meppel. Daarnaast ben ik flexibel en kom ik ook graag bij jou langs (reiskosten in overleg).<\\/p> \\n<p>Een behandeling is \u20ac30 Inclusief kopje thee en wat lekkers. De duur van een healing is gemiddeld een uur. Je kan zelf een tijdstip uitkiezen. Trek daarna

In [5]:
def outdoors_vs_tech(event_stream):
    try:
        event = json.loads(event_stream)
        description = event.get('description')
        group_cat = event.get('group')
        if group_cat:
            group_cat = group_cat.get('category')
            

        if group_cat == 'outdoors/adventure':
            return [description, 0]
        elif group_cat == 'tech':
            return [description, 1]
        else:
            pass
    except ValueError:
        pass

In [6]:
def status_binarizer(event_stream):
    try:
        event = json.loads(event_stream)
        description = event.get('description')
        status = event.get('status')            

        if status == 'upcoming':
            return [description, 0]
        elif status == 'suggested':
            return [description, 1]
        else:
            pass
    except ValueError:
        pass

In [7]:
events_df = events.map(status_binarizer).filter(\
    lambda x: x is not None).toDF(['description', 'status']).cache()

In [16]:
tokenizer = RegexTokenizer(inputCol="event", outputCol="words", pattern='\s+|[,.\"]')
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=200)
idf = IDF(inputCol="rawFeatures", outputCol="features")
RFC = RandomForestClassifier(labelCol="status", featuresCol="features", numTrees=10)

pipeline = Pipeline(stages=[\
                tokenizer,
                hashingTF,
                idf,
                forestizer])

In [17]:
pipeline.getStages()

[RegexTokenizer_464da418ce87ea834f42,
 HashingTF_4bd4b8891f99aea03bdd,
 IDF_447fb54cf2bff7f0060b,
 RandomForestClassifier_41a0ba488bd84de52726]

In [18]:
events_df_train, events_df_test = events_df.randomSplit([0.7, 0.3], seed=123)

In [19]:
model = pipeline.fit(events_df_train) 

IllegalArgumentException: u'Field "event" does not exist.'